# ORDINARY KRIGING - 1D - Single Point

**Acknowledgements: ** This Notebook is based on an original blogpost by Connor Johnson, 
http://connor-johnson.com/2014/03/20/simple-kriging-in-python/.

In that blogpost, the data set used (ZoneA.dat) was available from Geoffrey C. Bohling's webpage hosted by the University of Kansas. It does not seem to be available from there any longer. The version of the data that is provided in this repository (kriglab) was downloaded from a link from a notebook (https://colab.research.google.com/github/agile-geoscience/xlines/blob/master/notebooks/11_Gridding_map_data.ipynb#scrollTo=tKrxguz-vpI_) by Matt Hall (https://www.linkedin.com/in/geoscientist/).

## Original Post

In this post I will work through an example of Simple Kriging. Kriging is a set of techniques for interpolation. It differs from other interpolation techniques in that it sacrifices smoothness for the integrity of sampled points. Most interpolation techniques will over or undershoot the value of the function at sampled locations, but kriging honors those measurements and keeps them fixed. In future posts I would like to cover other types of kriging, other semivariaogram models, and colocated co-kriging. Until then, I’m keeping relatively up to date code at my GitHub project, geostatsmodels. (https://github.com/cjohnson318/geostatsmodels)

In [ ]:
from pylab import *
import numpy as np
from pandas import DataFrame, Series
from scipy.spatial.distance import pdist, squareform
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [16 / 1.5, 10 / 1.5]   # inch / cm = 2.54
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
# plt.rcParams['savefig.frameon'] = False

## Loading the Data

The data used in this exercise is in a zip file at this site. (Click on Geostatistics Resources.) I have adapted my material from the Kriging document on the same site. My approach will focus more on programming. First we will import some modules and then load the data and parse it,

In [ ]:
zraw = open( 'example_data/ZoneA.dat','r' ).readlines()
zraw = [ i.strip().split() for i in zraw[10:] ]
zraw = np.array( zraw, dtype=np.float )
zraw = DataFrame( zraw, columns=['x','y','thk','por','perm','lperm','lpermp','lpermr'] )

### Choose a sub-set and reduce to Quasi-1D

For this exercise, we need to reduce the data, and only choose values within a certain intervall for y, and create a quasi-1D Dataset by zeroing the y-coordinate. This is necessary as the method may fail during matrix inversion if there are two points with (near) equal x-coordinates, leading to a singular matrix.

In [ ]:
# choose the data subset from a y-intervall
ymean = 7500
yinterval = 2000

z = zraw  # create copy of full data set

# show the intervall before reducing the data
fig, ax = subplots()
ax.scatter( z.x, z.y, c=z.por, cmap='gray' )
ax.plot((-1500,22000),(ymean - yinterval / 2, ymean - yinterval / 2), "r--")
ax.plot((-1500,22000),(ymean + yinterval / 2, ymean + yinterval / 2), "r--")
ax.plot((-1500,22000),(ymean, ymean), "r-")
ax.set_aspect(1)
xlim(-1500,22000)
ylim(-1500,17500)
xlabel('Easting [m]')
ylabel('Northing [m]')
title('Porosity %') ;

plt.show()

z = z.loc[z.y > ymean - yinterval / 2]
z = z.loc[z.y < ymean + yinterval / 2]


In [ ]:
# reduce to quasi 1-D by assigning zero as y-coordinates for all samples,
# and sort the array by x-coordinate for better plottability.

z.y = 0.
z = z.sort_values(by="x")

plt.subplot(211)
plt.scatter( z.x, z.y, c=z.por, cmap='gray' )
ylabel('Northing [m]')
title('Porosity %') ;

plt.subplot(212)
plt.plot(z.x, z.por, ".--")
ylabel('Value')
xlabel('Easting [m]')
plt.show()

# part of our data set recording porosity
P = np.array( z[['x','y','por']] )
print("P =")
print(P)


## The Semivariogram

The semivariogram encodes data about spatial variance over the region at a given distance or lag. We generally expect data points that are close together spatially to share other characteristics, and we expect points that are separated by greater distances to have lesser correlation. The semivariogram allows us to model the similarity points in a field as a function of distance. The semivariogram is given by,

$$\begin{equation*} \hat{\gamma}(h) = \dfrac{1}{2N(h)} \displaystyle \sum_{N(h)} ( z_{i} - z_{j} )^{2} \end{equation*}$$ (Equation 1)

Here, h is distance specified by the user, and z_{i} and z_{j} are two points that are separated spatially by h. The N(h) term is the number of points we have that are separated by the distance h. The semivariogram then is the sum of squared differences between values separated by a distance h. As an aside, contrast this with the formulation for variance,

$$\begin{equation*} s = \dfrac{1}{N-1} \displaystyle \sum_{k=1}^{N} (z_{k} - \hat{\mu} )^{2} \end{equation*}$$ (Equation 2)

Here, $N$ is the number of data points, $\hat{\mu}$ is the sample mean, and $z_{k}$ is a data point. For sample variance, we are taking the squared difference between data points and the mean, and in the semivariogram we are taking the squared difference between data points separated by distance h. We can write some functions to calculate the semivariogram at one lag, and then at multiple lags as follows.

In [ ]:
def SVh( P, h, bw ):
    '''
    Experimental semivariogram for a single lag
    '''
    pd = squareform( pdist( P[:,:2] ) )
    N = pd.shape[0]
    Z = list()
    for i in range(N):
        for j in range(i+1,N):
            if( pd[i,j] >= h-bw )and( pd[i,j] <= h+bw ):
                Z.append( ( P[i,2] - P[j,2] )**2.0 )
    return np.sum( Z ) / ( 2.0 * len( Z ) )
 
def SV( P, hs, bw ):
    '''
    Experimental variogram for a collection of lags
    '''
    sv = list()
    for h in hs:
        sv.append( SVh( P, h, bw ) )
    sv = [ [ hs[i], sv[i] ] for i in range( len( hs ) ) if sv[i] > 0 ]
    return np.array( sv ).T
 
def C( P, h, bw ):
    '''
    Calculate the sill
    '''
    c0 = np.var( P[:,2] )
    if h == 0:
        return c0
    return c0 - SVh( P, h, bw )

The C() function is the covariance function, and will be used later. Let us now calculate and plot the semivariogram,

In [ ]:
# bandwidth, plus or minus 250 meters
bw = 500
# lags in 500 meter increments from zero to 10,000
hs = np.arange(0,10500,bw)
sv = SV( P, hs, bw )
plot( sv[0], sv[1], '.-' )
xlabel('Lag [m]')
ylabel('Semivariance')
title('Empirical Semivariogram') ;
savefig('sample_semivariogram.png', fmt='png', dpi=200)

Modeling
Now that we’ve calculated the semivariogram, we will need to fit a model to the data. There are three popular models, the spherical, exponential, and the Gaussian. Here, we’ll implement the spherical model. First, we will present a function named opt() for determining the optimal value a for the spherical model.

In [ ]:
def opt( fct, x, y, C0, parameterRange=None, meshSize=1000 ):
    if parameterRange == None:
        parameterRange = [ x[1], x[-1] ]
    mse = np.zeros( meshSize )
    a = np.linspace( parameterRange[0], parameterRange[1], meshSize )
    for i in range( meshSize ):
        mse[i] = np.mean( ( y - fct( x, a[i], C0 ) )**2.0 )
    return a[ mse.argmin() ]

The opt() function finds the optimal parameter for fitting a spherical model to the semivariogram data. The spherical model is given by the function spherical(). On the last line we see that spherical() returns itself in a map() function, which seems odd. The idea is that the input h can be a single float value, or list or NumPy array of floats. If h is a single value, then line 9 is called. If h is a list or an array (an iterable) then line 17 is called, which applies line 9 to each value of h.

In [ ]:
def spherical( h, a, C0 ):
    '''
    Spherical model of the semivariogram
    '''
    # if h is a single digit
    if type(h) == np.float64:
        # calculate the spherical function
        if h <= a:
            return C0*( 1.5*h/a - 0.5*(h/a)**3.0 )
        else:
            return C0
    # if h is an iterable
    else:
        # calcualte the spherical function for all elements
        a = np.ones( h.size ) * a
        C0 = np.ones( h.size ) * C0
        return list(map( spherical, h, a, C0 ))

Next, cvmodel() fits a model to the semivariogram data and returns a covariance method named covfct().

In [ ]:
def cvmodel( P, model, hs, bw ):
    '''
    Input:  (P)      ndarray, data
            (model)  modeling function
                      - spherical
                      - exponential
                      - gaussian
            (hs)     distances
            (bw)     bandwidth
    Output: (covfct) function modeling the covariance
    '''
    # calculate the semivariogram
    sv = SV( P, hs, bw )
    # calculate the sill
    C0 = C( P, hs[0], bw )
    # calculate the optimal parameters
    param = opt( model, sv[0], sv[1], C0 )
    # return a covariance function
    covfct = lambda h, a=param: C0 - model( h, a, C0 )
    return covfct

At this point we’ll plot our model and see if it represents our data well.

In [ ]:
sp = cvmodel( P, model=spherical, hs=np.arange(0,10500,500), bw=500 )
C0 = C( P, hs[0], bw )
plot( sv[0], sv[1], '.-' )
plot( sv[0], C0 - sp( sv[0] ) ) ;
title('Spherical Model')
ylabel('Semivariance')
xlabel('Lag [m]')
savefig('semivariogram_model.png',fmt='png',dpi=200)


## Ordinary Kriging

*from simple kriging:*

Now that we have a model for the semivariogram, we can write a function to perform the kriging. The fundamental relationship is a matrix equation,

$$\begin{equation*} K w = k \Rightarrow w = K^{-1} k \end{equation*} $$ (Equation 1)

Here, $K$ is a matrix of covariances of sampled data points, calculated using the spherical model, $\lambda$ is a vector of simple kriging weights, and $k$ is the vector of covariances between the data points and an unsampled point. Our kriging function takes the data set $P$, the model, the distances hs, the bandwidth bw, the coordinates of the unsampled point u, and the number of surrounding points N to use in the calculation.

In terms of ordinary kriging (OK) also a constant mean is assumed, but this time it is assumed to be unknown and only constant for local nearby estimation points. Due to that the sum of the kriging weights w has to be one.

$$ \hat{Z}_{OK}(u) = \Sigma^N_{i=1} w_i Z(u_i)$$ with $$  \Sigma^N_{i=1} w_i = 1$$

As already explained in 3.2 the aim with kriging is to minimize the estimation variance. This aim to-gether with the unbiasedness constrain that the weights have to sum up to one, leads to the need of a Lagrange multiplier $\lambda$ when using ordinary kriging. The Lagrangian multiplier method helps to solve an optimization problem (in our case the minimization of the estimator variance) with auxilia-ry conditions (weights have to sum up to one). Written in matrix and vector form, the ordinary kriging equation can then be formulated as:

$$ \boldsymbol{K} = 
\begin{bmatrix} \boldsymbol{K_1} & \boldsymbol{1}\\ 1 & 0 \end{bmatrix}
\begin{bmatrix} \boldsymbol{w_1}\\ \lambda \end{bmatrix}=
\begin{bmatrix} \boldsymbol{k_0}\\ 1 \end{bmatrix}$$

with

$$ K_1 = \begin{bmatrix} C_{11} & \dots & C_{n1}\\
\vdots & \ddots & \vdots\\
C_{1n} & \dots & C_{nn} \end{bmatrix} $$

$$ w_1 = \begin{bmatrix} w_1\\
\vdots \\
w_n\end{bmatrix} $$

$$ k_0 = \begin{bmatrix} c_{10}\\
\vdots \\
c_{n0}\end{bmatrix} $$

The first term is the covariance matrix K of equation 1, the second the weight vector w and on the right hand side of the equal sign the covariance vector k.
To solve the equation for the weights and lambda:

$$ \begin{bmatrix} \boldsymbol{w_1}\\ \lambda \end{bmatrix} =
\begin{bmatrix} \boldsymbol{K_1} & \boldsymbol{1}\\ 1 & 0 \end{bmatrix}^{-1}
\begin{bmatrix} \boldsymbol{k_0}\\ 1 \end{bmatrix}$$

### Choosing the parameters

In [ ]:
# h, k, K, weights = krige( P, sp, (dy*j,dx*i), len(z) / 2 )       
# PARAMETER DEFINITION
#           (P)     data set [y,x,value]
#           (model) modeling function
#                    - spherical
#                    - exponential
#                    - gaussian
#           (u)     unsampled point
#           (N)     number of neighboring
#                   points to consider

# P - Data set already defined
covfct = sp  # covariance function
u = (12000, 0)  # (x,y) - coordinates of unsampled points
N = 5  # number of neighbors to consider
assert N < len(P) + 1, "Number of Neighbors greater than number of data points"

### Calculating the Kriging Weights

#### Distance Vector (Unsampled Point <-> Sampled Points)

The distance between the unsampled point $u$ and each data point in $P$ is calculated, creating a vector $d$ of size 1xM, where M is the number of data points.

This vector becomes an additional column to the matrix $P$, which allows to sort the rows of $P$ by their distance to the data point, and only selecting to closest N neighbors.. 

In [ ]:
# distance between u and each data point in P (1xN vector)
d = np.sqrt( ( P[:,0]-u[0] )**2.0 + ( P[:,1]-u[1] )**2.0 )  # 2D 
print("d = "+str(d))

In [ ]:
# add these distances to P
P_n = np.vstack(( P.T, d )).T
# sort P by these distances
# take the first N of them
P_n = P_n[d.argsort()[:N]]
print("P_n = \n" + str(P_n))

#### Covariance Vector $\boldsymbol{k_0}$

For each of the remaining points, the expected covariance between the point and the unsampled point $u$ is calculated based on its distance by the convariance function

$$\boldsymbol{k_0} = cov(\boldsymbol{d})$$

In [ ]:
# apply the covariance model to the distances
k = covfct( P_n[:,3] )
# cast as a matrix
k = np.matrix( k ).T
print("k =\n" + str(k))

#### Covariance Matrix $\boldsymbol{K_1}$

To create the covariance matrix $K_1$ with the covariances of all sampled points, the same function is being used:

$$K_1 = [cov(d_{ij})]$$

with $d_{ij}$ being the Euclidian distance between points $i$ and $j$.

In [ ]:
# form a matrix of distances between existing data points
K_1 = squareform( pdist( P_n[:,:2] ) )
# apply the covariance model to these distances
K_1 = covfct( K_1.ravel() )
# re-cast as a NumPy array -- thanks M.L.
K = np.array( K_1 )
# reshape into an array
K_1 = K_1.reshape(N,N)
# cast as a matrix
K_1 = np.matrix( K_1 )
print("K_1 =\n"+str(K_1))

#### Introducting the LaGrange Factor

Ordinary Kriging requires the Matrix equation system to be extended

$$ \boldsymbol{K} = 
\begin{bmatrix} \boldsymbol{K_1} & \boldsymbol{1}\\ 1 & 0 \end{bmatrix}$$

In [ ]:
# Introducing the LaGrange Factor
K_line1 = np.hstack([K_1 , np.ones((K_1.shape[0],1))])
K_line2 = np.hstack([np.ones((1, K_1.shape[0])),np.zeros((1,1))])
K = np.vstack([K_line1, K_line2])
print("K =\n"+str(K_1))

$$k = \begin{bmatrix} \boldsymbol{k_0}\\ 1 \end{bmatrix}$$

In [ ]:
k = np.vstack([k,1])
print("k =\n" + str(k))

#### Solving for the weights

this allows calculating the Kriging weights $w$ and LaGrange factor $\lambda$ as defined previously

$$ \begin{bmatrix} \boldsymbol{w_1}\\ \lambda \end{bmatrix} =
\begin{bmatrix} \boldsymbol{K_1} & \boldsymbol{1}\\ \boldsymbol{1} & 0 \end{bmatrix}^{-1}
\begin{bmatrix} \boldsymbol{k_0}\\ 1 \end{bmatrix}$$

In [ ]:
# calculate the kriging weights
lhs = np.linalg.inv( K ) * k
lhs = np.array( lhs )

weights = lhs[:-1]
lagrange = weights[-1]

print("weights =\n"+str(weights))
print("laGrange =\n"+str(lagrange))

### Calculating the Prediction and Variance

Ordinary Kriging:

$$o = P[:,2]$$

$$e(u) =w^T \cdot o $$

--- muss nochmal geprüft werden, vermutlich muss der LaGrange faktor noch addiert werde ---

with $o$ being the vector of observed values at the sampling points, and $e(u)$ the estimate at the unsampled location $u$.

In [ ]:
o = np.array(P_n[:,2])
print("observations = "+str(o))

In [ ]:
# calculate the estimation
estimation = np.dot( weights.T, o )
print("estimation = "+str(estimation))

$$\sigma^2 = C_0 - w^T \cdot k$$

In [ ]:
variance = C0 - np.dot( weights.T, k[:-1] )
print("variance = "+str(variance))

## Visualize the Result

The plot below shows the estimate (error bar equals +/- 1 standard deviation) and the sampled points. For the {{N}} samples closest to the estimate the kriging weights are shown.  

In [ ]:
plt.plot(z.x, z.por, 
         "o--", label="samples")
plt.scatter((u[0]), (estimation), 
            c="red", label="estimate")
plt.errorbar((u[0]), (estimation), yerr = sqrt(float(variance)), 
             fmt="r", label="standard deviation")

offset = 0.05
for i in range(len(weights)):
    plt.text(P_n[i,0]+offset, 
             P_n[i,2]+offset, 
             "{:.2f}".format(float(weights[i])))

ylabel('Value')
xlabel('Easting [m]')
plt.legend()
plt.show()